# Make .batch files for the culster
by: Kristine M. L. Krighaar


In [1]:
import os
import numpy as np

def write_batch_file(folder, filename_prefix, value1, value2):
    # Construct the file path
    value2 = str(value2).replace(".", "p")
    file_path = os.path.join(folder, f"{filename_prefix}_{value1}_{value2}.batch")
    

    # Write the batch file content
    with open(file_path, 'w') as file:
        file.write(f'#!/bin/bash\n')
        file.write(f'\n')
        file.write(f'#SBATCH --job-name=BIFROST_{filename_prefix}_{value1}_{value2}\n')
        file.write(f'#SBATCH --error=err_BIFROST_{filename_prefix}_{value1}_{value2}.stderr\n')
        file.write(f'#SBATCH --output=out_BIFROST_{filename_prefix}_{value1}_{value2}.stdout\n')
        file.write(f'#SBATCH --nodes 1-1\n')
        file.write(f'#SBATCH --partition quark\n')
        file.write(f'#SBATCH --time=23:59:00\n')
        file.write(f'# the --exclusive is needed when running OpenMPI\n')
        file.write(f'# it will allocate 1x12 core per node\n')
        file.write(f'#SBATCH --exclusive\n')
        file.write(f'\n')
        file.write(f'NUMCORES=`echo "$SLURM_NNODES 56 * p "| dc`\n')
        file.write(f'\n')
        file.write(f'source /etc/profile.d/modules.sh\n')
        file.write(f'module avail\n')
        file.write(f'module purge\n')
        file.write(f'export OMPI_MCA_btl=^vader,tcp,openib\n')
        file.write(f'module load mcstas/3.4\n')
        file.write(f'module load gcc/10.2.0\n')
        file.write(f'module load openmpi/4.0_gcc1020\n')
        file.write(f'\n')
        file.write(f'### Set up mcrun line\n')
        file.write(f'\n')
        file.write(f'mcrun -c --mpi=$NUMCORES backend_Pb_1t.instr -d {filename_prefix}_{value1}_{value2}  A4={value1}  A3={value2}\n')




# Example usage
output_folder = 'Phonon_Pb_v1'
A4 = [60,65]
A3 = np.arange(10, 100, 0.5, dtype='float')  # Example list of values for value2

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Generate batch files for all combinations of values1 and values2
for v1 in A4:
    for v2 in A3:
        #print(v1,v2)
        write_batch_file(output_folder, 'Phonon_Pb_v1', v1, v2)


# Make the .sh script which will call all the .batch scripts on the cluster

In [3]:
def append_file_names(folder):
    # Get list of files in the folder
    file_names = os.listdir(folder)
    
    # Construct the output file path
    output_file_path = os.path.join(folder, 'one_script_to_rule_them_all.sh')
    
    # Open the output file and write the specific first line
    with open(output_file_path, 'w') as output:  # Use 'w' mode to overwrite the file
        output.write("#!/bin/bash\n")
    
    # Get list of files in the folder
    file_names = os.listdir(folder)

    # Append file names to the output file
    with open(output_file_path, 'a') as output:
        for file_name in file_names:
            if file_name != 'one_script_to_rule_them_all.sh':  # Exclude the output file itself
                output.write(f'sbatch {file_name}\n')

# Call the function to append file names to the output file
append_file_names(output_folder)